Импорт библиотек

In [2]:
import pdfplumber
import pandas as pd

Создание парсера данных из .pdf файла

In [6]:
df = pd.DataFrame(columns={'Index': [], 'Фамилия' : [], 'Имя и отчество': [], 'Дата рождения': []})
with pdfplumber.open('15-soldiers_of_victory_15.pdf') as pdf:
    count = 0
    for page in pdf.pages[73:]:
        # конвертируем текст из файла
        text = page.extract_text()
        tokens = text.split(' ')
        # начинаем поиск элементов по ключевому слову "род."
        indexes = [i for i, c in enumerate(tokens) if c == 'род.']
        for i in indexes:
            last_name = None
            for t in tokens[i:]:
                # определяем фамилию (может подобрать неправильно из-за неправильного формата .pdf файла)
                last_name = [t for t in tokens[:i] if t.endswith('ОВ*')]
                if len(last_name) >= 1:
                    true_name = last_name[-1]
                    if '\n' in true_name:
                        true_name = true_name[true_name.index('\n') + 1:]
                    last_name = true_name.replace('*', '')
                else: last_name = 'undefined'
                # имя и отчество
                almost_full_name = ' '.join(tokens[i-2:i])
                if '\n' in almost_full_name:
                    almost_full_name = almost_full_name[almost_full_name.index('\n') + 1:]
                almost_full_name = almost_full_name.replace(',', '')
                if '1' in t and len(t) == 4:
                    # определяем год, заносим найденные результаты в датафрейм и прерываем итерацию
                    df.loc[len(df.index)] = [count, last_name, almost_full_name, t]
                    count += 1
                    break
    print(df)
    df.to_csv('parsed_data.csv', index=False)

      Index      Фамилия         Имя и отчество Дата рождения
0         0      АКСЁНОВ       Павел Трофимович          1895
1         1  АЛЕКСАНДРОВ        Яков Михайлович          1908
2         2  АЛЕКСАНДРОВ       Николай Иванович          1925
3         3      АКСЁНОВ       Фёдор Трофимович          1911
4         4      АКСЁНОВ  Сергей Константинович          1990
...     ...          ...                    ...           ...
2348   2348        РУФОВ        Николай Кузьмич          1908
2349   2349    undefined      Пётр Владимирович          1911
2350   2350       ИВАНОВ       Пётр Григорьевич          1908
2351   2351       ИВАНОВ     Николай Алексеевич          1925
2352   2352       ИВАНОВ   Александр Филиппович          1943

[2353 rows x 4 columns]
